In [1]:
!pip install --upgrade gspread gspread_dataframe oauth2client

In [2]:
!pip install muq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 133.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 114.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 127.7 MB/s eta 0:00:00


In [5]:
# Authenticate with Google in Colab
from google.colab import auth
auth.authenticate_user()

# Authorize with gspread
import gspread
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)

# Open Google Sheet by URL
sh = gc.open_by_url("https://docs.google.com/spreadsheets/d/1DG3jWi9nWZ2hEzOsO5t6xtNgsmDJ8sdKOmGxJZKkMl8/edit#gid=1706329183")

worksheet = sh.worksheet("MuQMuLan")

In [6]:
from transformers import ClapModel, AutoProcessor
import torch
import librosa
import os
import pandas as pd
import numpy as np

In [7]:
from muq import MuQMuLan

# This will automatically fetch checkpoints from huggingface
device = 'cpu'
mulan = MuQMuLan.from_pretrained("OpenMuQ/MuQ-MuLan-large")
mulan = mulan.to(device).eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/847 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


pytorch_model.bin:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.65G [00:00<?, ?B/s]

# Process audio

In [8]:
import zipfile
import os

zip_files = ["Exp1.zip", "Exp2.zip", "Analysis.zip"]

extract_dir = "/content/" # You can change this if you want to extract elsewhere
os.makedirs(extract_dir, exist_ok=True)

for zip_file in zip_files:
    if os.path.exists(zip_file):
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
        print(f"Extracted {zip_file} to {extract_dir}")
    else:
        print(f"Error: {zip_file} not found.")

Extracted Exp1.zip to /content/
Extracted Exp2.zip to /content/
Extracted Analysis.zip to /content/


In [9]:
embeddings = []
stimuli_path = "/content/Exp1/Stimuli/"

for file in sorted(os.listdir(stimuli_path)):
    if file.endswith(".wav"):
        wav_path = os.path.join(stimuli_path, file)
        audio, sample_rate = librosa.load(wav_path, sr=24000)
        wav_tensor = torch.tensor(audio).unsqueeze(0).to(device)
        with torch.no_grad():
            embedding = mulan(wavs = wav_tensor)
        embeddings.append(embedding)

In [10]:
audio_embeddings = torch.stack(embeddings)
audio_embeddings = audio_embeddings.squeeze(1)
audio_embeddings.shape

torch.Size([59, 512])

# Process text

In [11]:
discrete_tags = ["happiness", "sadness", "anger", "tenderness", "fear"]

discrete_captions_perceived = [tag for tag in discrete_tags]
print(discrete_captions_perceived)
discrete_captions_induced = [tag for tag in discrete_tags]
print(discrete_captions_induced)

dimensional_tags = ["positive", "relaxed", "awake", "like"]

dimensional_captions_perceived = [tag for tag in dimensional_tags]
print(dimensional_captions_perceived)
dimensional_captions_induced = [tag for tag in dimensional_tags]
print(dimensional_captions_induced)


['happiness', 'sadness', 'anger', 'tenderness', 'fear']
['happiness', 'sadness', 'anger', 'tenderness', 'fear']
['positive', 'relaxed', 'awake', 'like']
['positive', 'relaxed', 'awake', 'like']


In [12]:
with torch.no_grad():
    tag_embeds = mulan(texts = dimensional_captions_induced)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [13]:
with torch.no_grad():
    tag_embeds_disc_i = mulan(texts = discrete_captions_induced)

In [14]:
with torch.no_grad():
    tag_embeds_dim_p = mulan(texts = dimensional_captions_perceived)

In [15]:
with torch.no_grad():
    tag_embeds_disc_p = mulan(texts = discrete_captions_perceived)

## Load csv files and extract related columns

In [16]:
IDim_path = '/content/Exp1/Data/IDim/'
IDim_response_dfs = []

for file in sorted(os.listdir(IDim_path)):
    if file.endswith(".csv"):
        file_path = os.path.join(IDim_path, file)
        try:
            df = pd.read_csv(file_path, sep=r'\s*,\s*', engine='python')
            # Crucial: Strip whitespace from column names
            df.columns = df.columns.str.strip()

            # Ensure required rating columns exist
            required_cols = ['positive', 'relaxed', 'awake','like']
            if all(col in df.columns for col in required_cols):
                # Select only the relevant columns and append to our list
                IDim_response_dfs.append(df[required_cols])
            else:
                print(f"Skipping file '{file_path}': Missing required columns ({required_cols}). Found columns: {df.columns.tolist()}")

        except Exception as e:
            print(f"Error reading or processing file {file_path}: {e}")


# Concatenate all individual DataFrames into one master DataFrame for human responses
if IDim_response_dfs:
    master_human_responses_df = pd.concat(IDim_response_dfs, ignore_index=True)
    print(f"Master human responses DataFrame shape: {master_human_responses_df.shape}\n")
    print(f"Master human responses (first 5 rows):\n{master_human_responses_df.head()}\n")
else:
    raise ValueError("No valid CSV files found or processed in IDim_path.")

Master human responses DataFrame shape: (3835, 4)

Master human responses (first 5 rows):
   positive  relaxed  awake  like
0      3.68     3.78   4.42  3.41
1      5.88     5.98   3.89  5.54
2      6.53     5.59   6.59  6.17
3      6.26     5.71   6.88  6.18
4      2.80     2.62   5.15  1.87



In [17]:
PDim_path = '/content/Exp1/Data/PDim/'
PDim_response_dfs = []

for file in sorted(os.listdir(PDim_path)):
    if file.endswith(".csv"):
        file_path = os.path.join(PDim_path, file)
        try:
            df = pd.read_csv(file_path, sep=r'\s*,\s*', engine='python')
            # Crucial: Strip whitespace from column names
            df.columns = df.columns.str.strip()

            # Ensure required rating columns exist
            required_cols = ['positive', 'relaxed', 'awake', 'like']
            if all(col in df.columns for col in required_cols):
                # Select only the relevant columns and append to our list
                PDim_response_dfs.append(df[required_cols])
            else:
                print(f"Skipping file '{file_path}': Missing required columns ({required_cols}). Found columns: {df.columns.tolist()}")

        except Exception as e:
            print(f"Error reading or processing file {file_path}: {e}")


# Concatenate all individual DataFrames into one master DataFrame for human responses
if PDim_response_dfs:
    master_human_responses_df_dim_p = pd.concat(PDim_response_dfs, ignore_index=True)
    print(f"Master human responses DataFrame shape: {master_human_responses_df_dim_p.shape}\n")
    print(f"Master human responses (first 5 rows):\n{master_human_responses_df_dim_p.head()}\n")
else:
    raise ValueError("No valid CSV files found or processed in IDim_path.")

Master human responses DataFrame shape: (3953, 4)

Master human responses (first 5 rows):
   positive  relaxed  awake  like
0      2.23     9.00   2.37  1.68
1      6.56     5.64   4.12  7.52
2      5.11     6.23   4.04  6.72
3      7.14     7.67   2.95  7.41
4      1.85     1.77   2.98  1.40



In [18]:
IDisc_path = '/content/Exp1/Data/IDisc/'
IDisc_response_dfs = []

for file in sorted(os.listdir(IDisc_path)):
    if file.endswith(".csv"):
        file_path = os.path.join(IDisc_path, file)
        try:
            df = pd.read_csv(file_path, sep=r'\s*,\s*', engine='python')
            # Crucial: Strip whitespace from column names
            df.columns = df.columns.str.strip()

            # Ensure required rating columns exist
            required_cols = ["happiness", "sadness", "anger", "tenderness", "fear"]
            if all(col in df.columns for col in required_cols):
                # Select only the relevant columns and append to our list
                IDisc_response_dfs.append(df[required_cols])
            else:
                print(f"Skipping file '{file_path}': Missing required columns ({required_cols}). Found columns: {df.columns.tolist()}")

        except Exception as e:
            print(f"Error reading or processing file {file_path}: {e}")


# Concatenate all individual DataFrames into one master DataFrame for human responses
if IDisc_response_dfs:
    master_human_responses_df_disc_i = pd.concat(IDisc_response_dfs, ignore_index=True)
    print(f"Master human responses DataFrame shape: {master_human_responses_df_disc_i.shape}\n")
    print(f"Master human responses (first 5 rows):\n{master_human_responses_df_disc_i.head()}\n")
else:
    raise ValueError("No valid CSV files found or processed in IDim_path.")

Master human responses DataFrame shape: (3894, 5)

Master human responses (first 5 rows):
   happiness  sadness  anger  tenderness  fear
0       1.00     6.97   5.01        9.00  7.89
1       1.00     6.00   5.80        6.78  1.00
2       1.00     2.42   5.99        6.44  1.09
3       3.92     6.13   2.25        5.96  1.22
4       1.99     7.92   6.82        5.83  1.39



In [19]:
PDisc_path = '/content/Exp1/Data/PDisc/'
PDisc_response_dfs = []

for file in sorted(os.listdir(PDisc_path)):
    if file.endswith(".csv"):
        file_path = os.path.join(PDisc_path, file)
        try:
            df = pd.read_csv(file_path, sep=r'\s*,\s*', engine='python')
            # Crucial: Strip whitespace from column names
            df.columns = df.columns.str.strip()

            # Ensure required rating columns exist
            required_cols = ["happiness", "sadness", "anger", "tenderness", "fear"]
            if all(col in df.columns for col in required_cols):
                # Select only the relevant columns and append to our list
                PDisc_response_dfs.append(df[required_cols])
            else:
                print(f"Skipping file '{file_path}': Missing required columns ({required_cols}). Found columns: {df.columns.tolist()}")

        except Exception as e:
            print(f"Error reading or processing file {file_path}: {e}")


# Concatenate all individual DataFrames into one master DataFrame for human responses
if PDisc_response_dfs:
    master_human_responses_df_disc_p = pd.concat(PDisc_response_dfs, ignore_index=True)
    print(f"Master human responses DataFrame shape: {master_human_responses_df_disc_p.shape}\n")
    print(f"Master human responses (first 5 rows):\n{master_human_responses_df_disc_p.head()}\n")
else:
    raise ValueError("No valid CSV files found or processed in IDim_path.")

Master human responses DataFrame shape: (3835, 5)

Master human responses (first 5 rows):
   happiness  sadness  anger  tenderness  fear
0       1.32     1.30   1.27        1.25  3.18
1       1.00     1.44   1.76        1.00  1.24
2       1.00     5.87   1.23        1.28  1.56
3       1.92     3.36   4.82        1.20  2.32
4       1.23     2.31   7.10        1.61  2.86



# Prepare features X and targets y

In [21]:
from sklearn.model_selection import train_test_split
"""
For each participant:
1. Takes all 59 audio embeddings
2. Pairs them with that participant's 59 ratings
3. Stacks these pairs for all participants.
"""

# Checks the number of participants is an exact multiple of the number of audio files (ensuring every participant rated every stimuli)
num_participants = len(IDim_response_dfs)
if master_human_responses_df.shape[0] % len(embeddings) != 0:
    print("Warning: Total responses is not a perfect multiple of unique audio files. This might indicate inconsistent data or that not all participants rated all items, which could break implicit ordering.")

# Audio embeddings generated by CLAP
# Creates one copy of all embeddings per participant
X_list = []
for _ in range(num_participants):
    X_list.extend(audio_embeddings.detach()) # Add a full set of embeddings for each participant

# Convert to NumPy array
X = np.array(X_list)

# Extract y from the concatenated DataFrame
y = master_human_responses_df[['positive', 'relaxed', 'awake', 'like']].values

print(f"Shape of X (features) after implicit alignment: {X.shape}")
print(f"Shape of y (labels) after implicit alignment: {y.shape}\n")

# Sanity check: X and y must have the same number of rows
# Critical check that we have exactly one set of features per label
if X.shape[0] != y.shape[0]:
    raise ValueError("Number of rows in X and y do not match after implicit alignment. This indicates an issue with the implicit ordering assumption or data loading.")

# --- Split Data into Training and Testing Sets ---
# 80% for training, 20% for testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

"""
Example:
Result (for 2 participants × 59 files = 118 total samples):
1. X_train: 94 audio embeddings (80% of 118)
2. y_train: 94 corresponding rating vectors
3. X_test: 24 audio embeddings (20%)
4. y_test: 24 rating vectors
'''

# X_train = combination of audio embeddings stacked on top of eachother
# y_train = combination of participants ratings for an audio file stacked ontop of eachother
# x_test = audio embedding for one audio file only
# y_test = participant rating for one audio file only

# Goal: check if x_test and y_test match

print(f"Training set size (X_train, y_train): {X_train.shape}, {y_train.shape}")
print(f"Testing set size (X_test, y_test): {X_test.shape}, {y_test.shape}\n")

In this example:
y_train = [
    [5.0, 4.0, 6.0],  # P1-A
    [7.0, 6.0, 4.0],   # P1-C
    [2.0, 1.0, 6.0],   # P2-B
    [4.0, 3.0, 5.0],   # P2-A
    [6.0, 7.0, 3.0]    # P2-C
]

X_test = [
    [0.5, 0.6, 0.7, 0.8]  # B.wav (P1)
]  # 1 sample

y_test = [
    [3.0, 2.0, 5.0]  # P1-B
]
- Training:
  X_train[0] = [0.1, 0.2, 0.3, 0.4] → Predict y_train[0] = [5.0, 4.0, 6.0]
"""

Shape of X (features) after implicit alignment: (3835, 512)
Shape of y (labels) after implicit alignment: (3835, 4)



'\nExample:\nResult (for 2 participants × 59 files = 118 total samples):\n1. X_train: 94 audio embeddings (80% of 118)\n2. y_train: 94 corresponding rating vectors\n3. X_test: 24 audio embeddings (20%)\n4. y_test: 24 rating vectors\n\'\'\'\n\n# X_train = combination of audio embeddings stacked on top of eachother\n# y_train = combination of participants ratings for an audio file stacked ontop of eachother\n# x_test = audio embedding for one audio file only\n# y_test = participant rating for one audio file only\n\n# Goal: check if x_test and y_test match\n\nprint(f"Training set size (X_train, y_train): {X_train.shape}, {y_train.shape}")\nprint(f"Testing set size (X_test, y_test): {X_test.shape}, {y_test.shape}\n")\n\nIn this example:\ny_train = [\n    [5.0, 4.0, 6.0],  # P1-A\n    [7.0, 6.0, 4.0],   # P1-C\n    [2.0, 1.0, 6.0],   # P2-B\n    [4.0, 3.0, 5.0],   # P2-A\n    [6.0, 7.0, 3.0]    # P2-C\n]\n\nX_test = [\n    [0.5, 0.6, 0.7, 0.8]  # B.wav (P1)\n]  # 1 sample\n\ny_test = [\n   

In [22]:
from sklearn.model_selection import train_test_split

num_participants_dim_p = len(PDim_response_dfs)
if master_human_responses_df_dim_p.shape[0] % len(embeddings) != 0:
    print("Warning: Total responses is not a perfect multiple of unique audio files. This might indicate inconsistent data or that not all participants rated all items, which could break implicit ordering.")

X_list = []
for _ in range(num_participants_dim_p):
    X_list.extend(audio_embeddings.detach()) # Add a full set of embeddings for each participant

# Convert to NumPy array
X_dim_p = np.array(X_list)

# Extract y from the concatenated DataFrame
y_dim_p = master_human_responses_df_dim_p[['positive', 'relaxed', 'awake', 'like']].values

print(f"Shape of X (features) after implicit alignment: {X_dim_p.shape}")
print(f"Shape of y (labels) after implicit alignment: {y_dim_p.shape}\n")

# Sanity check: X and y must have the same number of rows
if X_dim_p.shape[0] != y_dim_p.shape[0]:
    raise ValueError("Number of rows in X and y do not match after implicit alignment. This indicates an issue with the implicit ordering assumption or data loading.")

# --- Split Data into Training and Testing Sets ---
X_train_dim_p, X_test_dim_p, y_train_dim_p, y_test_dim_p = train_test_split(
    X_dim_p, y_dim_p, test_size=0.2, random_state=42
)

print(f"Training set size (X_train, y_train): {X_dim_p.shape}, {y_train_dim_p.shape}")
print(f"Testing set size (X_test, y_test): {X_test_dim_p.shape}, {y_test_dim_p.shape}\n")

Shape of X (features) after implicit alignment: (3953, 512)
Shape of y (labels) after implicit alignment: (3953, 4)

Training set size (X_train, y_train): (3953, 512), (3162, 4)
Testing set size (X_test, y_test): (791, 512), (791, 4)



In [23]:
from sklearn.model_selection import train_test_split

num_participants_disc_i = len(IDisc_response_dfs)
if master_human_responses_df_disc_i.shape[0] % len(embeddings) != 0:
    print("Warning: Total responses is not a perfect multiple of unique audio files. This might indicate inconsistent data or that not all participants rated all items, which could break implicit ordering.")

X_list = []
for _ in range(num_participants_disc_i):
    X_list.extend(audio_embeddings.detach()) # Add a full set of embeddings for each participant

# Convert to NumPy array
X_disc_i = np.array(X_list)

# Extract y from the concatenated DataFrame
y_disc_i = master_human_responses_df_disc_i[["happiness", "sadness", "anger", "tenderness", "fear"]].values

print(f"Shape of X (features) after implicit alignment: {X_disc_i.shape}")
print(f"Shape of y (labels) after implicit alignment: {y_disc_i.shape}\n")

# Sanity check: X and y must have the same number of rows
if X_disc_i.shape[0] != y_disc_i.shape[0]:
    raise ValueError("Number of rows in X and y do not match after implicit alignment. This indicates an issue with the implicit ordering assumption or data loading.")

# --- Split Data into Training and Testing Sets ---
X_train_disc_i, X_test_disc_i, y_train_disc_i, y_test_disc_i = train_test_split(
    X_disc_i, y_disc_i, test_size=0.2, random_state=42
)

print(f"Training set size (X_train, y_train): {X_disc_i.shape}, {y_train_disc_i.shape}")
print(f"Testing set size (X_test, y_test): {X_test_disc_i.shape}, {y_test_disc_i.shape}\n")

Shape of X (features) after implicit alignment: (3894, 512)
Shape of y (labels) after implicit alignment: (3894, 5)

Training set size (X_train, y_train): (3894, 512), (3115, 5)
Testing set size (X_test, y_test): (779, 512), (779, 5)



In [24]:
from sklearn.model_selection import train_test_split

num_participants_disc_p = len(PDisc_response_dfs)
if master_human_responses_df_disc_p.shape[0] % len(embeddings) != 0:
    print("Warning: Total responses is not a perfect multiple of unique audio files. This might indicate inconsistent data or that not all participants rated all items, which could break implicit ordering.")

X_list = []
for _ in range(num_participants_disc_p):
    X_list.extend(audio_embeddings.detach()) # Add a full set of embeddings for each participant

# Convert to NumPy array
X_disc_p = np.array(X_list)

# Extract y from the concatenated DataFrame
y_disc_p = master_human_responses_df_disc_p[["happiness", "sadness", "anger", "tenderness", "fear"]].values

print(f"Shape of X (features) after implicit alignment: {X_disc_p.shape}")
print(f"Shape of y (labels) after implicit alignment: {y_disc_p.shape}\n")

# Sanity check: X and y must have the same number of rows
if X_disc_p.shape[0] != y_disc_p.shape[0]:
    raise ValueError("Number of rows in X and y do not match after implicit alignment. This indicates an issue with the implicit ordering assumption or data loading.")

# --- Split Data into Training and Testing Sets ---
X_train_disc_p, X_test_disc_p, y_train_disc_p, y_test_disc_p = train_test_split(
    X_disc_p, y_disc_p, test_size=0.2, random_state=42
)

print(f"Training set size (X_train, y_train): {X_disc_p.shape}, {y_train_disc_p.shape}")
print(f"Testing set size (X_test, y_test): {X_test_disc_p.shape}, {y_test_disc_p.shape}\n")

Shape of X (features) after implicit alignment: (3835, 512)
Shape of y (labels) after implicit alignment: (3835, 5)

Training set size (X_train, y_train): (3835, 512), (3068, 5)
Testing set size (X_test, y_test): (767, 512), (767, 5)



# Train regression head (=MLP, a few projection layers)

In [25]:
from sklearn.neural_network import MLPRegressor

mlp_regressor = MLPRegressor(
    hidden_layer_sizes=(100, 50),
    activation='relu',
    solver='adam',
    max_iter=1000,
    random_state=42,
    verbose=True,
    early_stopping=True,
    n_iter_no_change=50,
    tol=1e-4
)

print("Starting MLP Regressor training...")
mlp_regressor.fit(X_train, y_train)
print("\nMLP Regressor training complete.")

Starting MLP Regressor training...
Iteration 1, loss = 12.90124187
Validation score: -4.824940
Iteration 2, loss = 8.41015162
Validation score: -1.867792
Iteration 3, loss = 3.76289880
Validation score: -0.381641
Iteration 4, loss = 2.40541145
Validation score: -0.015950
Iteration 5, loss = 1.95174927
Validation score: 0.062005
Iteration 6, loss = 1.88206869
Validation score: 0.097140
Iteration 7, loss = 1.82646593
Validation score: 0.120715
Iteration 8, loss = 1.78879474
Validation score: 0.134872
Iteration 9, loss = 1.76237892
Validation score: 0.149908
Iteration 10, loss = 1.73707705
Validation score: 0.162541
Iteration 11, loss = 1.71315135
Validation score: 0.176105
Iteration 12, loss = 1.69158852
Validation score: 0.186258
Iteration 13, loss = 1.67362438
Validation score: 0.197731
Iteration 14, loss = 1.65464625
Validation score: 0.205231
Iteration 15, loss = 1.63906390
Validation score: 0.214318
Iteration 16, loss = 1.62302132
Validation score: 0.222196
Iteration 17, loss = 1.61

In [26]:
from sklearn.neural_network import MLPRegressor

mlp_regressor_dim_p = MLPRegressor(
    hidden_layer_sizes=(100, 50),
    activation='relu',
    solver='adam',
    max_iter=1000,
    random_state=42,
    verbose=True,
    early_stopping=True,
    n_iter_no_change=50,
    tol=1e-4
)

print("Starting MLP Regressor training...")
mlp_regressor_dim_p.fit(X_train_dim_p, y_train_dim_p)
print("\nMLP Regressor training complete.")

Starting MLP Regressor training...
Iteration 1, loss = 12.76673768
Validation score: -3.657901
Iteration 2, loss = 7.96338223
Validation score: -1.170803
Iteration 3, loss = 3.51344828
Validation score: -0.213900
Iteration 4, loss = 2.42923399
Validation score: 0.052195
Iteration 5, loss = 2.08861015
Validation score: 0.098965
Iteration 6, loss = 2.00804845
Validation score: 0.132465
Iteration 7, loss = 1.95262917
Validation score: 0.148203
Iteration 8, loss = 1.91678413
Validation score: 0.160722
Iteration 9, loss = 1.89080525
Validation score: 0.172435
Iteration 10, loss = 1.86510194
Validation score: 0.182696
Iteration 11, loss = 1.84136020
Validation score: 0.190067
Iteration 12, loss = 1.82169404
Validation score: 0.197572
Iteration 13, loss = 1.80564550
Validation score: 0.202708
Iteration 14, loss = 1.78679244
Validation score: 0.208552
Iteration 15, loss = 1.77228073
Validation score: 0.213250
Iteration 16, loss = 1.76071072
Validation score: 0.218297
Iteration 17, loss = 1.748

In [27]:
from sklearn.neural_network import MLPRegressor

mlp_regressor_disc_i = MLPRegressor(
    hidden_layer_sizes=(100, 50),
    activation='relu',
    solver='adam',
    max_iter=1000,
    random_state=42,
    verbose=True,
    early_stopping=True,
    n_iter_no_change=50,
    tol=1e-4
)

print("Starting MLP Regressor training...")
mlp_regressor_disc_i.fit(X_train_disc_i, y_train_disc_i)
print("\nMLP Regressor training complete.")

Starting MLP Regressor training...
Iteration 1, loss = 4.78410542
Validation score: -0.624923
Iteration 2, loss = 3.19153785
Validation score: -0.128732
Iteration 3, loss = 2.43262287
Validation score: 0.008860
Iteration 4, loss = 2.31450771
Validation score: 0.028078
Iteration 5, loss = 2.25135834
Validation score: 0.042164
Iteration 6, loss = 2.20338323
Validation score: 0.061752
Iteration 7, loss = 2.16255692
Validation score: 0.070608
Iteration 8, loss = 2.14425606
Validation score: 0.076278
Iteration 9, loss = 2.13359100
Validation score: 0.079647
Iteration 10, loss = 2.11995540
Validation score: 0.076965
Iteration 11, loss = 2.12510493
Validation score: 0.078883
Iteration 12, loss = 2.11069340
Validation score: 0.080573
Iteration 13, loss = 2.10338172
Validation score: 0.082309
Iteration 14, loss = 2.09973243
Validation score: 0.083866
Iteration 15, loss = 2.09200777
Validation score: 0.083243
Iteration 16, loss = 2.08828274
Validation score: 0.085155
Iteration 17, loss = 2.08946

In [28]:
from sklearn.neural_network import MLPRegressor

mlp_regressor_disc_p = MLPRegressor(
    hidden_layer_sizes=(100, 50),
    activation='relu',
    solver='adam',
    max_iter=1000,
    random_state=42,
    verbose=True,
    early_stopping=True,
    n_iter_no_change=50,
    tol=1e-4
)

print("Starting MLP Regressor training...")
mlp_regressor_disc_p.fit(X_train_disc_p, y_train_disc_p)
print("\nMLP Regressor training complete.")

Starting MLP Regressor training...
Iteration 1, loss = 6.88934085
Validation score: -1.040928
Iteration 2, loss = 4.83578172
Validation score: -0.311733
Iteration 3, loss = 3.29176958
Validation score: -0.013404
Iteration 4, loss = 2.79441789
Validation score: 0.040336
Iteration 5, loss = 2.69314103
Validation score: 0.078585
Iteration 6, loss = 2.59400960
Validation score: 0.110291
Iteration 7, loss = 2.51480731
Validation score: 0.137068
Iteration 8, loss = 2.46931084
Validation score: 0.149944
Iteration 9, loss = 2.44298798
Validation score: 0.152438
Iteration 10, loss = 2.42183620
Validation score: 0.160565
Iteration 11, loss = 2.40706639
Validation score: 0.160165
Iteration 12, loss = 2.39152864
Validation score: 0.166395
Iteration 13, loss = 2.37683187
Validation score: 0.166774
Iteration 14, loss = 2.36350687
Validation score: 0.169318
Iteration 15, loss = 2.35236916
Validation score: 0.170053
Iteration 16, loss = 2.34480472
Validation score: 0.169897
Iteration 17, loss = 2.3382

# Evaluate

In [29]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr

y_pred = mlp_regressor.predict(X_test)

print(f"\nShape of predictions (y_pred): {y_pred.shape}")
print(f"First 5 actual values (y_test):\n{y_test[:5]}")
print(f"First 5 predicted values (y_pred):\n{y_pred[:5]}\n")

# Evaluation Metrics:

# Mean Absolute Percentage Error (MAPE)
absolute_percentage_error = np.abs((y_test - y_pred) / y_test) * 100
mape = np.mean(absolute_percentage_error)
worksheet.update('E5', [[f"{mape:.2f}%"]])
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%\n")

# Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
worksheet.update('E2', [[f"{mae:.4f}"]])
print(f"Mean Absolute Error (MAE): {mae:.4f}")

# Root Mean Squared Error (RMSE)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
worksheet.update('E6', [[f"{rmse:.4f}"]])
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

# Pearson Correlation Coefficient (per dimension)
print("\nPearson Correlation Coefficients (per dimension):")
for i, dim_name in enumerate(['Positive', 'Relaxed', 'Awake', 'Like']):
    # Check for sufficient variance to calculate correlation
    if np.std(y_test[:, i]) > 1e-6 and np.std(y_pred[:, i]) > 1e-6:
        correlation, _ = pearsonr(y_test[:, i], y_pred[:, i])
        print(f"  {dim_name} Dimension: {correlation:.4f}")
    else:
        print(f"  {dim_name} Dimension: Cannot calculate (insufficient variance in data for this dimension)")

correlations = []
for i in range(y_test.shape[1]):
    if np.std(y_test[:, i]) > 1e-6 and np.std(y_pred[:, i]) > 1e-6:
        correlations.append(pearsonr(y_test[:, i], y_pred[:, i])[0])
if correlations:
    average_correlation = np.mean(correlations)
    print(f"  Average Pearson Correlation across dimensions: {average_correlation:.4f}")
    worksheet.update(
    range_name='E4',
    values=[[f"{average_correlation:.4f}"]]
    )
else:
    print("  No correlations could be calculated for averaging.")

correlation_string = ', '.join([f"{c:.4f}" for c in correlations])
# Update cell E3 with the full string
worksheet.update('E3', [[correlation_string]])

from sklearn.metrics import r2_score

# R-squared
print("\nR-squared scores:")
r2_valence = r2_score(y_test[:, 0], y_pred[:, 0])
print("  valence =", r2_valence)

r2_tension = r2_score(y_test[:, 1], y_pred[:, 1])
print("  tension =", r2_tension)

r2_energy = r2_score(y_test[:, 2], y_pred[:, 2])
print("  energy =", r2_energy)

r2_like = r2_score(y_test[:, 3], y_pred[:, 3])
print("  like =", r2_like)

worksheet.update(
    range_name='E7',
    values=[[f"{r2_valence:.4f}, {r2_tension:.4f}, {r2_energy:.4f}, {r2_like:.4f}"]]
)



Shape of predictions (y_pred): (767, 4)
First 5 actual values (y_test):
[[3.96 8.32 7.05 8.  ]
 [7.11 7.01 8.11 7.01]
 [4.68 5.04 5.88 5.1 ]
 [2.08 4.04 7.6  1.26]
 [6.34 6.43 5.32 4.8 ]]
First 5 predicted values (y_pred):
[[5.4226336 5.520157  5.3202286 5.4217033]
 [4.8717675 5.071002  5.0296965 4.9316697]
 [3.5399454 3.013194  6.656212  3.1540082]
 [3.2745788 2.5529933 6.915473  2.7635856]
 [5.0141754 4.8970003 5.4059916 4.792697 ]]



/tmp/ipython-input-29-2279673651.py:15: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E5', [[f"{mape:.2f}%"]])


Mean Absolute Percentage Error (MAPE): 42.13%

Mean Absolute Error (MAE): 1.4001


/tmp/ipython-input-29-2279673651.py:20: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E2', [[f"{mae:.4f}"]])
/tmp/ipython-input-29-2279673651.py:25: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E6', [[f"{rmse:.4f}"]])


Root Mean Squared Error (RMSE): 1.7553

Pearson Correlation Coefficients (per dimension):
  Positive Dimension: 0.5419
  Relaxed Dimension: 0.6017
  Awake Dimension: 0.2222
  Like Dimension: 0.5618
  Average Pearson Correlation across dimensions: 0.4819


/tmp/ipython-input-29-2279673651.py:54: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E3', [[correlation_string]])



R-squared scores:
  valence = 0.2928203428515429
  tension = 0.3609951782031463
  energy = 0.04122039720592818
  like = 0.3155595986865588


{'spreadsheetId': '1DG3jWi9nWZ2hEzOsO5t6xtNgsmDJ8sdKOmGxJZKkMl8',
 'updatedRange': 'MuQMuLan!E7',
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [30]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr

y_pred_dim_p = mlp_regressor_dim_p.predict(X_test_dim_p)

print(f"\nShape of predictions (y_pred): {y_pred_dim_p.shape}")
print(f"First 5 actual values (y_test):\n{y_test_dim_p[:5]}")
print(f"First 5 predicted values (y_pred):\n{y_pred_dim_p[:5]}\n")

# Evaluation Metrics:

# Mean Absolute Percentage Error (MAPE)
absolute_percentage_error = np.abs((y_test_dim_p - y_pred_dim_p) / y_test_dim_p) * 100
mape_dim_p = np.mean(absolute_percentage_error)
worksheet.update('E12', [[f"{mape_dim_p:.2f}%"]])
print(f"Mean Absolute Percentage Error (MAPE): {mape_dim_p:.2f}%\n")

# Mean Absolute Error (MAE)
mae_dim_p = mean_absolute_error(y_test_dim_p, y_pred_dim_p)
worksheet.update('E9', [[f"{mae_dim_p:.4f}"]])
print(f"Mean Absolute Error (MAE): {mae_dim_p:.4f}")

# Root Mean Squared Error (RMSE)
rmse_dim_p = np.sqrt(mean_squared_error(y_test_dim_p, y_pred_dim_p))
worksheet.update('E13', [[f"{rmse_dim_p:.4f}"]])
print(f"Root Mean Squared Error (RMSE): {rmse_dim_p:.4f}")

# Pearson Correlation Coefficient (per dimension)
print("\nPearson Correlation Coefficients (per dimension):")
for i, dim_name in enumerate(['Positive', 'Relaxed', 'Awake', 'Like']):
    # Check for sufficient variance to calculate correlation
    if np.std(y_test_dim_p[:, i]) > 1e-6 and np.std(y_pred_dim_p[:, i]) > 1e-6:
        correlation, _ = pearsonr(y_test_dim_p[:, i], y_pred_dim_p[:, i])
        print(f"  {dim_name} Dimension: {correlation:.4f}")
    else:
        print(f"  {dim_name} Dimension: Cannot calculate (insufficient variance in data for this dimension)")

correlations = []
for i in range(y_test_dim_p.shape[1]):
    if np.std(y_test_dim_p[:, i]) > 1e-6 and np.std(y_pred_dim_p[:, i]) > 1e-6:
        correlations.append(pearsonr(y_test_dim_p[:, i], y_pred_dim_p[:, i])[0])
if correlations:
    average_correlation = np.mean(correlations)
    worksheet.update(
    range_name='E11',
    values=[[f"{average_correlation:.4f}"]]
    )
    print(f"  Average Pearson Correlation across dimensions: {average_correlation:.4f}")
else:
    print("  No correlations could be calculated for averaging.")

correlation_string_dim_p = ', '.join([f"{c:.4f}" for c in correlations])
# Update cell E3 with the full string
worksheet.update('E10', [[correlation_string_dim_p]])

from sklearn.metrics import r2_score

# R-squared
print("\nR-squared scores:")
r2_valence_dim_p = r2_score(y_test_dim_p[:, 0], y_pred_dim_p[:, 0])
print("  valence =", r2_valence_dim_p)

r2_tension_dim_p = r2_score(y_test_dim_p[:, 1], y_pred_dim_p[:, 1])
print("  tension =", r2_tension_dim_p)

r2_energy_dim_p = r2_score(y_test_dim_p[:, 2], y_pred_dim_p[:, 2])
print("  energy =", r2_energy_dim_p)

r2_like_dim_p = r2_score(y_test_dim_p[:, 3], y_pred_dim_p[:, 3])
print("  like =", r2_like_dim_p)

worksheet.update(
    range_name='E14',
    values=[[f"{r2_valence_dim_p:.4f}, {r2_tension_dim_p:.4f}, {r2_energy_dim_p:.4f}, {r2_like_dim_p:.4f}"]]
)


Shape of predictions (y_pred): (791, 4)
First 5 actual values (y_test):
[[3.16 4.31 2.02 5.18]
 [8.86 8.86 8.78 8.53]
 [8.14 5.35 3.35 7.12]
 [5.02 5.01 5.99 5.98]
 [5.04 4.31 6.31 5.45]]
First 5 predicted values (y_pred):
[[4.751629  3.988885  6.412105  4.035136 ]
 [6.7876782 5.9794197 6.1297207 6.405773 ]
 [4.56246   3.6208744 6.763582  3.860315 ]
 [5.2662463 4.9793887 5.8513803 5.280173 ]
 [3.4442155 3.791537  3.9928648 4.2343087]]

Mean Absolute Percentage Error (MAPE): 46.95%



/tmp/ipython-input-30-2848619493.py:15: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E12', [[f"{mape_dim_p:.2f}%"]])
/tmp/ipython-input-30-2848619493.py:20: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E9', [[f"{mae_dim_p:.4f}"]])


Mean Absolute Error (MAE): 1.4733


/tmp/ipython-input-30-2848619493.py:25: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E13', [[f"{rmse_dim_p:.4f}"]])


Root Mean Squared Error (RMSE): 1.8374

Pearson Correlation Coefficients (per dimension):
  Positive Dimension: 0.5671
  Relaxed Dimension: 0.5297
  Awake Dimension: 0.4318
  Like Dimension: 0.5847
  Average Pearson Correlation across dimensions: 0.5283


/tmp/ipython-input-30-2848619493.py:54: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E10', [[correlation_string_dim_p]])



R-squared scores:
  valence = 0.32029980712787953
  tension = 0.2738376328319281
  energy = 0.18064128332322904
  like = 0.336908205451934


{'spreadsheetId': '1DG3jWi9nWZ2hEzOsO5t6xtNgsmDJ8sdKOmGxJZKkMl8',
 'updatedRange': 'MuQMuLan!E14',
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [31]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr

y_pred_disc_i = mlp_regressor_disc_i.predict(X_test_disc_i)

print(f"\nShape of predictions (y_pred): {y_pred_disc_i.shape}")
print(f"First 5 actual values (y_test):\n{y_test_disc_i[:5]}")
print(f"First 5 predicted values (y_pred):\n{y_pred_disc_i[:5]}\n")

# Evaluation Metrics:

# Mean Absolute Percentage Error (MAPE)
absolute_percentage_error_disc_i = np.abs((y_test_disc_i - y_pred_disc_i) / y_test_disc_i) * 100
mape_disc_i = np.mean(absolute_percentage_error_disc_i)
worksheet.update('E19', [[f"{mape_disc_i:.2f}%"]])
print(f"Mean Absolute Percentage Error (MAPE): {mape_disc_i:.2f}%\n")

# Mean Absolute Error (MAE)
mae_disc_i = mean_absolute_error(y_test_disc_i, y_pred_disc_i)
worksheet.update('E16', [[f"{mae_disc_i:.4f}"]])
print(f"Mean Absolute Error (MAE): {mae_disc_i:.4f}")

# Root Mean Squared Error (RMSE)
rmse_disc_i = np.sqrt(mean_squared_error(y_test_disc_i, y_pred_disc_i))
worksheet.update('E20', [[f"{rmse_disc_i:.4f}"]])
print(f"Root Mean Squared Error (RMSE): {rmse_disc_i:.4f}")

# Pearson Correlation Coefficient (per dimension)
print("\nPearson Correlation Coefficients (per dimension):")
for i, dim_name in enumerate(['Happiness', 'Sadness', 'Anger', 'Tenderness', 'Fear']):
    # Check for sufficient variance to calculate correlation
    if np.std(y_test_disc_i[:, i]) > 1e-6 and np.std(y_pred_disc_i[:, i]) > 1e-6:
        correlation, _ = pearsonr(y_test_disc_i[:, i], y_pred_disc_i[:, i])
        print(f"  {dim_name} Dimension: {correlation:.4f}")
    else:
        print(f"  {dim_name} Dimension: Cannot calculate (insufficient variance in data for this dimension)")

correlations = []
for i in range(y_test_disc_i.shape[1]):
    if np.std(y_test_disc_i[:, i]) > 1e-6 and np.std(y_pred_disc_i[:, i]) > 1e-6:
        correlations.append(pearsonr(y_test_disc_i[:, i], y_pred_disc_i[:, i])[0])
if correlations:
    average_correlation = np.mean(correlations)
    worksheet.update(
        range_name='E18',
        values=[[f"{average_correlation:.4f}"]]
      )
    print(f"  Average Pearson Correlation across dimensions: {average_correlation:.4f}")
else:
    print("  No correlations could be calculated for averaging.")

correlation_string_disc_i = ', '.join([f"{c:.4f}" for c in correlations])
# Update cell E3 with the full string
worksheet.update('E17', [[correlation_string_disc_i]])

from sklearn.metrics import r2_score

# R-squared IDisc
print("\nR-squared scores (IDisc):")
r2_happiness_disc_i = r2_score(y_test_disc_i[:, 0], y_pred_disc_i[:, 0])
print("  happiness =", r2_happiness_disc_i)
r2_sadness_disc_i = r2_score(y_test_disc_i[:, 1], y_pred_disc_i[:, 1])
print("  sadness =", r2_sadness_disc_i)
r2_anger_disc_i = r2_score(y_test_disc_i[:, 2], y_pred_disc_i[:, 2])
print("  anger =", r2_anger_disc_i)
r2_tenderness_disc_i = r2_score(y_test_disc_i[:, 3], y_pred_disc_i[:, 3])
print("  tenderness =", r2_tenderness_disc_i)
r2_fear_disc_i = r2_score(y_test_disc_i[:, 4], y_pred_disc_i[:, 4])
print("  fear =", r2_fear_disc_i)

worksheet.update(
    range_name='E21',
    values=[[f"{r2_happiness_disc_i:.4f}, {r2_sadness_disc_i:.4f}, {r2_anger_disc_i:.4f}, {r2_tenderness_disc_i:.4f}, {r2_fear_disc_i:.4f}"]]
)


Shape of predictions (y_pred): (779, 5)
First 5 actual values (y_test):
[[1.   1.   1.   1.   1.  ]
 [4.23 6.82 5.75 5.13 5.69]
 [1.28 1.24 1.15 1.18 1.23]
 [1.   1.   2.4  1.   1.54]
 [1.36 4.73 1.07 3.37 1.94]]
First 5 predicted values (y_pred):
[[2.125743  3.0979793 2.368946  2.7306979 2.2844942]
 [2.2439716 2.7764156 3.8641694 2.9022465 2.6466484]
 [2.934299  2.2518308 1.9740628 2.8715448 2.3758333]
 [1.7076838 2.855471  5.114919  2.30393   3.5357597]
 [2.3634322 3.0146408 2.3980749 2.8077738 2.3953414]]

Mean Absolute Percentage Error (MAPE): 84.90%



/tmp/ipython-input-31-609622067.py:15: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E19', [[f"{mape_disc_i:.2f}%"]])
/tmp/ipython-input-31-609622067.py:20: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E16', [[f"{mae_disc_i:.4f}"]])


Mean Absolute Error (MAE): 1.6112
Root Mean Squared Error (RMSE): 2.0324

Pearson Correlation Coefficients (per dimension):
  Happiness Dimension: 0.4021
  Sadness Dimension: 0.2311
  Anger Dimension: 0.4954
  Tenderness Dimension: 0.2643
  Fear Dimension: 0.3097


/tmp/ipython-input-31-609622067.py:25: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E20', [[f"{rmse_disc_i:.4f}"]])


  Average Pearson Correlation across dimensions: 0.3405

R-squared scores (IDisc):
  happiness = 0.1603583386424543
  sadness = 0.051798145496183334
  anger = 0.23824919725827065
  tenderness = 0.06748924134245915
  fear = 0.09500547810849491


/tmp/ipython-input-31-609622067.py:54: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E17', [[correlation_string_disc_i]])


{'spreadsheetId': '1DG3jWi9nWZ2hEzOsO5t6xtNgsmDJ8sdKOmGxJZKkMl8',
 'updatedRange': 'MuQMuLan!E21',
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [32]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr

y_pred_disc_p = mlp_regressor_disc_p.predict(X_test_disc_p)

print(f"\nShape of predictions (y_pred): {y_pred_disc_p.shape}")
print(f"First 5 actual values (y_test):\n{y_test_disc_p[:5]}")
print(f"First 5 predicted values (y_pred):\n{y_pred_disc_p[:5]}\n")

# Evaluation Metrics:

# Mean Absolute Percentage Error (MAPE)
absolute_percentage_error_disc_p = np.abs((y_test_disc_p - y_pred_disc_p) / y_test_disc_p) * 100
mape_disc_p = np.mean(absolute_percentage_error_disc_p)
worksheet.update('E26', [[f"{mape_disc_p:.2f}%"]])
print(f"Mean Absolute Percentage Error (MAPE): {mape_disc_p:.2f}%\n")

# Mean Absolute Error (MAE)
mae_disc_p = mean_absolute_error(y_test_disc_p, y_pred_disc_p)
worksheet.update('E23', [[f"{mae_disc_p:.4f}"]])
print(f"Mean Absolute Error (MAE): {mae_disc_p:.4f}")

# Root Mean Squared Error (RMSE)
rmse_disc_p = np.sqrt(mean_squared_error(y_test_disc_p, y_pred_disc_p))
worksheet.update('E27', [[f"{rmse_disc_p:.4f}"]])
print(f"Root Mean Squared Error (RMSE): {rmse_disc_p:.4f}")

# Pearson Correlation Coefficient (per dimension)
print("\nPearson Correlation Coefficients (per dimension):")
for i, dim_name in enumerate(['Happiness', 'Sadness', 'Anger', 'Tenderness', 'Fear']):
    # Check for sufficient variance to calculate correlation
    if np.std(y_test_disc_p[:, i]) > 1e-6 and np.std(y_pred_disc_p[:, i]) > 1e-6:
        correlation, _ = pearsonr(y_test_disc_p[:, i], y_pred_disc_p[:, i])
        print(f"  {dim_name} Dimension: {correlation:.4f}")
    else:
        print(f"  {dim_name} Dimension: Cannot calculate (insufficient variance in data for this dimension)")

correlations = []
for i in range(y_test_disc_p.shape[1]):
    if np.std(y_test_disc_p[:, i]) > 1e-6 and np.std(y_pred_disc_p[:, i]) > 1e-6:
        correlations.append(pearsonr(y_test_disc_p[:, i], y_pred_disc_p[:, i])[0])
if correlations:
    average_correlation = np.mean(correlations)
    worksheet.update(
        range_name='E25',
        values=[[f"{average_correlation:.4f}"]]
      )
    print(f"  Average Pearson Correlation across dimensions: {average_correlation:.4f}")
else:
    print("  No correlations could be calculated for averaging.")

correlation_string_disc_p = ', '.join([f"{c:.4f}" for c in correlations])
# Update cell E3 with the full string
worksheet.update('E24', [[correlation_string_disc_p]])

from sklearn.metrics import r2_score

# R-squared PDisc
print("\nR-squared scores (PDisc):")
r2_happiness_disc_p = r2_score(y_test_disc_p[:, 0], y_pred_disc_p[:, 0])
print("  happiness =", r2_happiness_disc_p)
r2_sadness_disc_p = r2_score(y_test_disc_p[:, 1], y_pred_disc_p[:, 1])
print("  sadness =", r2_sadness_disc_p)
r2_anger_disc_p = r2_score(y_test_disc_p[:, 2], y_pred_disc_p[:, 2])
print("  anger =", r2_anger_disc_p)
r2_tenderness_disc_p = r2_score(y_test_disc_p[:, 3], y_pred_disc_p[:, 3])
print("  tenderness =", r2_tenderness_disc_p)
r2_fear_disc_p = r2_score(y_test_disc_p[:, 4], y_pred_disc_p[:, 4])
print("  fear =", r2_fear_disc_p)

worksheet.update(
    range_name='E28',
    values=[[f"{r2_happiness_disc_p:.4f}, {r2_sadness_disc_p:.4f}, {r2_anger_disc_p:.4f}, {r2_tenderness_disc_p:.4f}, {r2_fear_disc_p:.4f}"]]
)

average_r2 = (r2_valence + r2_energy + r2_tension) /3
worksheet.update('E8', [[f"{average_r2:.4f}"]])

average_r2_dim_p = (r2_valence_dim_p + r2_energy_dim_p + r2_tension_dim_p) /3
worksheet.update('E15', [[f"{average_r2_dim_p:.4f}"]])

average_r2_disc_i = (r2_happiness_disc_i + r2_sadness_disc_i + r2_anger_disc_i + r2_tenderness_disc_i + r2_fear_disc_i) / 5
worksheet.update('E22', [[f"{average_r2_disc_i:.4f}"]])

average_r2_disc_p = (r2_happiness_disc_p + r2_sadness_disc_p + r2_anger_disc_p + r2_tenderness_disc_p + r2_fear_disc_p) / 5
worksheet.update('E29', [[f"{average_r2_disc_p:.4f}"]])

print("\nAverage R-squared scores:")
print("IDim: ", average_r2)
print("PDim: ", average_r2_dim_p)
print("IDisc: ", average_r2_disc_i)
print("PDisc: ", average_r2_disc_p)


Shape of predictions (y_pred): (767, 5)
First 5 actual values (y_test):
[[2.   6.04 1.86 1.01 1.01]
 [1.   6.06 1.   2.99 1.  ]
 [1.   2.01 4.02 1.   1.  ]
 [1.81 5.81 4.99 3.35 1.61]
 [1.26 3.29 2.45 1.26 6.45]]
First 5 predicted values (y_pred):
[[2.1372156 5.286106  2.9632554 2.893774  3.3577678]
 [1.4602578 4.7760334 4.6102753 2.34082   4.2342706]
 [2.1739917 4.3441324 4.0611196 2.669255  4.5371914]
 [1.9065093 4.1148334 4.911532  2.2123957 5.5072675]
 [2.623721  4.430024  2.2876604 3.48973   2.9894736]]

Mean Absolute Percentage Error (MAPE): 87.01%



/tmp/ipython-input-32-1059293014.py:15: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E26', [[f"{mape_disc_p:.2f}%"]])
/tmp/ipython-input-32-1059293014.py:20: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E23', [[f"{mae_disc_p:.4f}"]])


Mean Absolute Error (MAE): 1.7902
Root Mean Squared Error (RMSE): 2.1962

Pearson Correlation Coefficients (per dimension):
  Happiness Dimension: 0.4026
  Sadness Dimension: 0.3592
  Anger Dimension: 0.4248
  Tenderness Dimension: 0.3301
  Fear Dimension: 0.4123


/tmp/ipython-input-32-1059293014.py:25: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E27', [[f"{rmse_disc_p:.4f}"]])


  Average Pearson Correlation across dimensions: 0.3858


/tmp/ipython-input-32-1059293014.py:54: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E24', [[correlation_string_disc_p]])



R-squared scores (PDisc):
  happiness = 0.1481843521718591
  sadness = 0.12816724083641717
  anger = 0.16955948455653824
  tenderness = 0.10748380960172199
  fear = 0.16374290188774254


/tmp/ipython-input-32-1059293014.py:77: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E8', [[f"{average_r2:.4f}"]])
/tmp/ipython-input-32-1059293014.py:80: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E15', [[f"{average_r2_dim_p:.4f}"]])
/tmp/ipython-input-32-1059293014.py:83: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('E22', [[f"{average_r2_disc_i:.4f}"]])
/tmp/ipython-input-32-1059293014.py:86: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, 


Average R-squared scores:
IDim:  0.2316786394202058
PDim:  0.2582595744276789
IDisc:  0.12258008016957248
PDisc:  0.1434275578108558
